## 데이터 크롤링

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from time import sleep

 
# txt파일을 넣을 폴더를 생성한다.
song_folder_path = "./song_folder"
if not os.path.exists(song_folder_path):
    os.makedirs(song_folder_path)
 
 

headers = {
    'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                   '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36')
}
 
age_url = "https://www.melon.com/chart/age/list.htm"
 
params = {
    'idx': '1',
    'chartType': 'YE',     # 10년 단위로 검색하는 부분과 연관
    'chartGenre': 'KPOP',  # 가요검색: KPOP, 팝송검색: POP
    'chartDate': '1991',   # 검색연도
    'moved': 'Y',
}

 
response = requests.get(age_url, params=params, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')
song_list = soup.select('.lst50')
 
for i, meta in enumerate(song_list,1):
 
    ### 순위, 제목
    rank = i
    try:
        title = meta.select('a[href*=playSong]')[0].text
    except:
        title = meta.select('.wrap_song_info .ellipsis')[0].text
    title = title.strip()
      
    print(str(rank )+ '위. ', title)
 
 
    # 노래 데이터 url의 html 추출
    song_id_html = str(meta.select('a[onclick*=SongDetail]'))
    matched = re.search(r"\'(\d+)\'", song_id_html)
    song_id = matched.group(1)
    front_url = 'https://www.melon.com/song/detail.htm?songId='
    song_url = front_url + song_id
 
 
    ### 가수, 앨범명, 발매날짜, 장르
    response = requests.get(song_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    singer_html = soup.select('.wrap_info .artist a')
 
    ### 가수
    singer_s = []
    if len(singer_html) != 0:
        for html in singer_html:
            singer_s.append(html['title'])
    else:
        # url 없는 Various Artists용
        singer_html = str(soup.select('.wrap_info .artist')[0])
        singer_html = singer_html.replace('\t','').replace('\r','').split('\n')
        singer_html = ''.join(singer_html)
        matched = re.search(r">(.*)<", singer_html)
        singer_s.append(matched.group(1))
 
    # 가수가 여러명일 때 하나의 string으로 표현
    singer_s = ', '.join(singer_s)
 
 
    # 앨범명
    album_name_html = str(soup.select('.list dd')[0])
    matched = re.search(r">(.*)<", album_name_html)
    matched2 = re.search(r">(.*)<", matched.group(1))
    album_name = matched2.group(1).strip()
 
 
    # 발매날짜
    song_date_html = str(soup.select('.list dd')[1])
    matched = re.search(r">(.*)<", song_date_html)
    song_date = matched.group(1)
 
 
    # 장르
    song_genre_html = str(soup.select('.list dd')[2])
    matched = re.search(r">(.*)<", song_genre_html)
    song_genre = matched.group(1)
 
    print("가수:", singer_s)
    print("장르:", song_genre)
 
 
    ### 가사가 있으면 추출
    try:
        lyric_html = str(soup.select('.section_lyric .wrap_lyric .lyric')[0])
        lyric_html = lyric_html.replace('\t','').replace('\r','').split('\n')
        lyric_html = ''.join(lyric_html)
 
        matched = re.search(r"-->(.*)<br/>", lyric_html)
        lyric = matched.group(1).strip()
        lyric = lyric.replace('<br/>', '\n')
 
        #가사 앞뒤 빈칸 제거
        lyric_list = []
        for line in lyric.split('\n'):
            lyric_list.append(line.strip())
        lyric = ('\n').join(lyric_list)
 
    except:
        lyric = "없음"
 
 
    ### 작사가, 작곡가, 편곡가
    # 작사, 작곡, 편곡이 자리가 정해지지 않고, 존재하는 데이터만 앞쪽으로 밀려서 나온다.
    song_members = []
    for i in range(0,3):
        try:
            song_member = str(soup.select('.section_prdcr .list_person .entry .artist')[i].text).strip()
            member_data = str(soup.select('.section_prdcr .list_person .entry .meta')[i].text).strip()
            song_members.append([song_member, member_data])
        except:
            pass
 
    composer = ""
    lyricist = ""
    arranger = ""
    for data in song_members:
        if data[1] == '작사':
            composer = data[0]
        if data[1] == '작곡':
            lyricist = data[0]
        if data[1] == '편곡':
            arranger = data[0]
 
    print("작사가:",lyricist)
    print()
 
    print('(가사)',sep='\n')
    print(lyric)
    print()
 
 
    #파일이름 불가 문자제거
    title = re.sub(r"[\/\:\*\?\"\<\>\|]", " ", title)
 
    ### 노래 txt 추출
    file_name = str(params['chartDate']) + '_' + str(rank) + '. ' + singer_s + ' - ' + title + ".txt"
    txt_path = song_folder_path + "/" + file_name
    
    #인코딩 판단
    try:
        f = open(txt_path, 'w')
        f.write(lyric)
    except:
        f = open(txt_path, 'w', encoding='utf-8')
        f.write(lyric)
    f.close()
 
    # IP 차단 방지용
    sleep(1)



